### Code Credits
The majority of this code comes from a script written by Ira Koroleva on the Conservation Technology Team:
https://github.com/tnc-ca-geo/freshwater/blob/master/Function_loading_layers_from_AGOL.ipynb

In [ ]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import datetime as dt
from datetime import date
import getpass
from shapely.validation import explain_validity
from shapely.validation import make_valid

In [ ]:
def get_AGOL_layer(layer_id, crs, output_file=None): 
    """
    This function loads the layer from ArcGIS Online, converts it to pandas geodataframe, projects, 
    and saves to disc as a new file with current date in the name.
    
    Parameters:
    layer_id* (str): ArcGIS Online layer ID can be retrieved from its browser link. For example, in 
    "https://tnc.maps.arcgis.com/home/item.html?id=23fcdb5591ae4889a054b63bcbd7fc98" the ID comes after "id=" and 
    is equal to 23fcdb5591ae4889a054b63bcbd7fc98.
    
    crs* (int): geographic coordinate system as EPSG code. You can find the codes at https://spatialreference.org/ref/epsg/
    
    output_file (str): optional; full path to the shapefile wiht extension ".shp" where the output file will be 
    saved. Please add "r" before the string or use double slash "\\" instead of single slash in the path.
    
    Example:
    tnc = get_AGOL_layer(layer_id = 'de8681a325f643f49a1fc848a0dac5bb', 
                        crs = 3310,
                        output_file = r'C:\Documents\TNC_lands.shp')
    
    Please address your questions to Ira Koroleva (irina.koroleva@tnc.org)
    The Nature Conservancy, 2023
    """
    
    # log into AGOL
    link = 'https://www.arcgis.com/sharing/rest'
    gis = GIS(link, client_id='ztKl8vv9x4R8bE20')
    
    # Get the hosted feature layer
    flayer = gis.content.get(layer_id).layers[0]
    # .query() returns a FeatureSet
    fset = flayer.query()
    # Get a GeoJSON string representation of the FeatureSet
    gjson_string = fset.to_geojson
    # Read GeoJSON string into a dict
    gjson_dict = json.loads(gjson_string)
    # Read in as geodataframe
    layer = gpd.GeoDataFrame.from_features(gjson_dict['features'], crs=3857)
    
    # Project layer
    layer = layer.to_crs(crs)
    
    # Check validity
    total = len(layer['geometry'])
    valid = len(layer.is_valid)
    invalid = total - valid
    
    # Multipolygons might have been turned into polygons and their parts became invalid, fix them by turning them back into multipolygons
    if invalid > 0:
        layer['geometry'] = layer['geometry'].apply(make_valid)
        print(invalid + ' invalid features were fixed')
    
    # Save as shapefile if the location is provided
    if not (output_file is None):
        # Change any date fields to text
        layer_save = layer.copy()
        cols = list(layer_save)
        for i in cols:
            if layer_save[i].dtype == 'datetime64[ns]': 
                layer_save[i] = layer_save[i].astype(str)
        
        layer_save.to_file(output_file) 
    return layer

In [ ]:
# Example - loading TNC lands
tnc = get_AGOL_layer(layer_id = 'f77e630cfe914483929f2a0bfba230c3', 
                    crs = 4326,
                    output_file = r'C:\Users\jinsu.elhance\file.shp');